In [1]:
%pylab widget

import copy
from matplotlib import animation

In [4]:
class Particles:
    def __init__(self, n, g=1e-5, softening=0.1, boxsize=1, periodic=False, seed=12345):
        #book-keeping
        self.n=n
        self.g=g
        self.softening=softening
        self.boxsize=1
        self.periodic=periodic
        #initialize randomly with seed 
        self.rand=np.random.default_rng(seed)
        self.X=self.rand.uniform(size=[n, 2])
        self.V=np.zeros_like(self.X)
        self.a=self.acc()
        
    def walk(self):
        self.X+=self.V+0.5*self.a
        self.V+=self.a
        self.a=self.acc()
        if self.periodic:
            self.X%=self.boxsize   
    
    def acc(self):
        boxhalf=self.boxsize*0.5
        a=np.zeros_like(self.X)
        for i in range(self.n-1):
            for j in range(i+1, self.n):
                dx=self.X[j]-self.X[i]
                if self.periodic:
                    dx%=self.boxsize
                    dx[dx<-boxhalf]+=self.boxsize
                    dx[dx>boxhalf]-=self.boxsize
                r=np.sqrt((dx**2).sum())
                a_ij=self.g*dx/(r+self.softening)**3
                a[i]+=a_ij
                a[j]+=-a_ij
        return a

In [5]:
def animate(particles):
    # First set up the figure
    fig = plt.figure()
    ax = plt.axes(xlim=(0, particles.boxsize), ylim=(0, particles.boxsize))
    p=copy.deepcopy(particles)
    line1, =ax.plot(p.X.T[0], p.X.T[1], 'bo', mfc='none')
    ax.set_title('t=0')
    
    def init():
        nonlocal p
        p=copy.deepcopy(particles) #reset p
        return line1

    #how to evolve the plot
    def draw_next(i):
        p.walk()
        line1.set_data(p.X.T[0], p.X.T[1])
        ax.set_title('t=%d'%i)
        return line1
    
    # call the animator.  blit=True means only re-draw the parts that have changed.
    movie = animation.FuncAnimation(fig, draw_next, init_func=init,
                                   frames=150, interval=10, blit=True)

    return movie

In [6]:
p=Particles(20, g=1e-5, softening=0.1, periodic=True ,seed=321)

In [7]:
m=animate(p)
#m.save('../plots/GravityDemo.gif', fps=10)#, extra_args=['-vcodec', 'libx264'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
m.save('../plots/GravityDemo.gif', fps=10)#, extra_args=['-vcodec', 'libx264'])

MovieWriter ffmpeg unavailable; using Pillow instead.
